In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random

from statistics import mean

In [2]:
class ThreeLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(ThreeLayerNet, self).__init__()
        
        # smaller layers
        half_H = int(H/2)
        quarter_H = int(H/4)
        
        # input and output connections for each layer
        linear_sizes = [[D_in, H], [H,H], [H, half_H], [half_H, D_out]]
        self.linear_layers = len(linear_sizes)
        
        # actual layer container
        self.layers = []        
        
        # create layers as we defined them before
        for i in range(self.linear_layers):
            s = linear_sizes[i]
            layer = torch.nn.Linear(s[0],s[1])
            self.layers.append(layer)
        
        # tell torch where our layers 
        # need this to be able to optimize them
        self.parameters = nn.ModuleList(self.layers)
        
        # loss function
        self.criterion = torch.nn.MSELoss()       
        
        # optimizer & learning rate
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    
    # for prediction
    def forward(self, x):
        # 
        for i in range(self.linear_layers):
            x = F.relu(self.layers[i](x))       
        return x 
    
    def train(self, x, y):   
        # predict value
        y_pred = self(x)        
        # get loss
        loss   = self.criterion(y_pred, y)        
        # prepare our output values
        output = loss.item(), y_pred, y
        
        # learn from error
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()    
        
        return output

In [3]:
# source: https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
# takes in a module and applies the specified weight initialization
def weights_init_uniform(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # apply a uniform distribution to the weights and a bias=0
        m.weight.data.uniform_(0, 1.0)
        m.bias.data.fill_(0)


In [4]:
# create model
# num features in, hidden layer size, num predictions out
model = ThreeLayerNet(2, 10, 2)

# randomize model weights
model.apply(weights_init_uniform)


ThreeLayerNet(
  (parameters): ModuleList(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): Linear(in_features=5, out_features=2, bias=True)
  )
  (criterion): MSELoss()
)

In [5]:
losses = []
count = 0
epochs = 1000
for epoch in range(epochs):
    for num in range(100):        
        # create 2 random numbers
        a = random.randint(0,10000)
        b = random.randint(0,10000)
        
        # turn them into a tensor
        features = torch.FloatTensor([a,b])
        truth = torch.FloatTensor([a,b])        
        
        # train network on its ability to predict them
        loss_val, y_pred, y = model.train(features, truth)
        
        # print loss to user
        count += 1
        if count % 5000 == 0:
            # keep track of last few output losses
            losses = losses[-3:]
            losses.append(loss_val)
            
            # convert prediction into easy to print list
            guess = [int(x) for x in y_pred.data.tolist()]
            
            # display true value, predictions, and loss to user. 
            print(count, "\ntrue: ", [a,b], "\nguess:", guess, "\nloss:", mean(losses), "\n")

5000 
true:  [4708, 5153] 
guess: [20804, 49337] 
loss: 1105683072.0 

10000 
true:  [6079, 7577] 
guess: [6560, 8961] 
loss: 553378984.75 

15000 
true:  [1256, 3574] 
guess: [2482, 2920] 
loss: 369241331.5416667 

20000 
true:  [8581, 9780] 
guess: [8759, 9599] 
loss: 276939063.7705078 

25000 
true:  [6832, 1312] 
guess: [6865, 1294] 
loss: 518479.47914123535 

30000 
true:  [1707, 3719] 
guess: [1704, 3720] 
loss: 249756.49104392529 

35000 
true:  [6647, 2417] 
guess: [6643, 2417] 
loss: 8251.452851057053 

40000 
true:  [8188, 2446] 
guess: [8191, 2448] 
loss: 188.28765332698822 

45000 
true:  [5075, 9870] 
guess: [5084, 9883] 
loss: 37.71104657649994 

50000 
true:  [9740, 7083] 
guess: [9744, 7073] 
loss: 49.80830240249634 

55000 
true:  [8624, 2730] 
guess: [8621, 2725] 
loss: 51.98738420009613 

60000 
true:  [9534, 365] 
guess: [9536, 365] 
loss: 50.69034427404404 

65000 
true:  [1509, 5967] 
guess: [1511, 5993] 
loss: 102.9842819571495 

70000 
true:  [4208, 4455] 
guess